In [20]:
from quantizeml.models import quantize, QuantizationParams
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from losses import PoseEstimationLoss

tf.keras.utils.get_custom_objects().update({"pose_estimation_loss": PoseEstimationLoss})
model = keras.models.load_model("2.15_latest_model.keras",custom_objects={"pose_estimation_loss": PoseEstimationLoss} )
print("Model loaded successfully!")
model.summary()

qparams = QuantizationParams(input_weight_bits=8, weight_bits=4, activation_bits=4)
# TODO: call quantize with the real training data
# https://doc.brainchipinc.com/examples/general/plot_0_global_workflow.html#sphx-glr-examples-general-plot-0-global-workflow-py
# model_quantized = quantize(model_keras, qparams=qparams,
                        #    samples=x_train, num_samples=1024, batch_size=100, epochs=2)
model_quantized = quantize(model, qparams=qparams)
model_quantized.summary()


TypeError: PoseEstimationLoss.__init__() got an unexpected keyword argument 'reduction'

In [ ]:
'''
# TODO: evaluate model
'''def compile_evaluate(model):
    """ Compiles and evaluates the model, then return accuracy score. """
    model.compile(metrics=['accuracy'])
    return model.evaluate(x_test, y_test, verbose=0)[1]



print('Test accuracy after 8-bit quantization:', compile_evaluate(model_quantized))

model_quantized.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy'])

# score = model_quantized.evaluate(x_test, y_test, verbose=0)[1]
# print('Test accuracy after fine tuning:', score)

# model_quantized.fit(x_train, y_train, epochs=5, validation_split=0.1)



### CONVERT TO AIKIDA NETWORK

from cnn2snn import convert

model_akida = convert(model_quantized)
model_akida.summary()


# # check performance
# accuracy = model_akida.evaluate(x_test, y_test)
# print('Test accuracy after conversion:', accuracy)

# # For non-regression purposes
# assert accuracy > 0.96''''''

In [3]:
import tensorflow as tf
from tensorflow import keras

# Load the model
try:
    model = keras.models.load_model("latest_model.h5")
    print("Model loaded successfully!")
    model.summary()
except Exception as e:
    print(f"Error loading the model:{e}")



Error loading the model:Could not interpret loss identifier: pose_estimation_loss


In [6]:
# Load weights into the model
weights_path = "latest_model.h5"
model.load_weights(weights_path)
print("Weights loaded successfully!")




NameError: name 'model' is not defined

In [ ]:
import os

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
base_model = keras.applications.MobileNet(include_top=False, alpha=0.7 ,input_shape=(240, 240, 3), weights=None)
# base_model.trainable = False  # Freeze all layers in the base model

x = base_model.output
x = tf.keras.layers.Flatten()(x)
output_l = tf.keras.layers.Dense(7, activation='linear')(x)
model_keras = Model(inputs=base_model.input, outputs=output_l)

print(model_keras.summary())

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 240, 240, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 120, 120, 22)   │           594 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 120, 120, 22)   │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 120, 120, 22)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 120, 120, 22)   │           198 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 120, 120, 22)   │            88 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 120, 120, 22)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 120, 120, 44)   │           968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 120, 120, 44)   │           176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 120, 120, 44)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 121, 121, 44)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 60, 60, 44)     │           396 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 60, 60, 44)     │           176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 60, 60, 44)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 60, 60, 89)     │         3,916 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 60, 60, 89)     │           356 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 60, 60, 89)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 60, 60, 89)     │           801 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 60, 60, 89)     │           356 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 60, 60, 89)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 60, 60, 89)     │         7,921 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 60, 60, 89)     │           356 │
│ (BatchNormalization)            │                        │             

 Total params: 1,842,638 (7.03 MB)

 Trainable params: 1,827,342 (6.97 MB)

 Non-trainable params: 15,296 (59.75 KB)

None


In [17]:
weights_path = '/home/linux_mj/hackathon/2024-nc-hackathon-spades/astrospikes/latest_model.h5'
model.load_weights(weights_path)
print("Weights loaded successfully!")

# Step 5: Check the model summary
model.summary()

ValueError: Weight count mismatch for layer #0 (named conv1 in the current model, conv1 in the save file). Layer expects 2 weight(s). Received 1 saved weight(s)

In [20]:
import h5py

def load_weights_manually(model, weights_path):
    with h5py.File(weights_path, 'r') as f:
        for layer in model.layers:
            if layer.name in f:
                layer_weights = f[layer.name]
                layer.set_weights([layer_weights[key] for key in layer_weights])
    print("Weights loaded successfully!")

# Use this function after defining your model
model = model_keras
load_weights_manually(model, "/home/linux_mj/hackathon/2024-nc-hackathon-spades/astrospikes/latest_model.h5")


Weights loaded successfully!


In [21]:
model.save("full_model.h5")


In [22]:
model = keras.models.load_model("full_model.h5")


In [23]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 240, 240, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 120, 120, 22)   │           594 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 120, 120, 22)   │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 120, 120, 22)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 120, 120, 22)   │           198 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 120, 120, 22)   │            88 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 120, 120, 22)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 120, 120, 44)   │           968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 120, 120, 44)   │           176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 120, 120, 44)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 121, 121, 44)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 60, 60, 44)     │           396 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 60, 60, 44)     │           176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 60, 60, 44)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 60, 60, 89)     │         3,916 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 60, 60, 89)     │           356 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 60, 60, 89)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 60, 60, 89)     │           801 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 60, 60, 89)     │           356 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 60, 60, 89)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 60, 60, 89)     │         7,921 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 60, 60, 89)     │           356 │
│ (BatchNormalization)            │                        │             

 Total params: 1,842,638 (7.03 MB)

 Trainable params: 1,827,342 (6.97 MB)

 Non-trainable params: 15,296 (59.75 KB)

In [27]:
# Check if weights were loaded correctly for a specific layer
conv1_weights = model.get_layer('conv1').get_weights()
print(f"Conv1 Layer Weights: {conv1_weights[0].shape}")


# If the weights are loaded correctly, the output will not be empty
if conv1_weights:
    print("Weights have been loaded for the 'conv1' layer.")
else:
    print("Weights not loaded correctly for the 'conv1' layer.")


Conv1 Layer Weights: (3, 3, 3, 22)
Weights have been loaded for the 'conv1' layer.


In [28]:
# Check if weights were loaded correctly for the 'conv1' layer
conv1_weights = model.get_layer('conv1').get_weights()

if conv1_weights:
    # Print the shape of the weights
    print(f"Conv1 Layer Weights Shape: {conv1_weights[0].shape}")
    
    # Calculate the sum of all weights (excluding biases)
    weights_sum = conv1_weights[0].sum()  # Sum of the weights
    print(f"Sum of Conv1 Layer Weights: {weights_sum}")
    
    # If biases exist, calculate their sum as well
    if len(conv1_weights) > 1:
        biases_sum = conv1_weights[1].sum()  # Sum of the biases
        print(f"Sum of Conv1 Layer Biases: {biases_sum}")
else:
    print("Weights not loaded correctly for the 'conv1' layer.")


Conv1 Layer Weights Shape: (3, 3, 3, 22)
Sum of Conv1 Layer Weights: 4.224855899810791
